In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, accuracy_score

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics.regression import MeanSquaredError, MeanAbsoluteError
from torchmetrics.classification import BinaryAUROC, BinaryAveragePrecision
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.callbacks import Callback

import optuna
from optuna.integration import PyTorchLightningPruningCallback

/data/miniconda3/envs/arcr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터셋 정의

In [2]:
DATA_MAKER = "jiseock"
if DATA_MAKER == "jiseock":
    DATA_PATH = "../dataset/jiseock"
else:
    DATA_PATH = "../dataset/yeonseo"

X_train = pd.read_csv(f"{DATA_PATH}/X_train.csv")
y_train = pd.read_csv(f"{DATA_PATH}/y_train.csv")

In [3]:
X_train

,나이,"성별 (M:1,F:2)","Rt:1,Lt:2",Height,Weight,"Tearsize (AP,cm)",Tearsize (ML),Tearsize (retraction),"흡연여부 (비흡연:1,흡연:2)","흡연여부 (비흡연:1,흡연:2) Missing flag",...,6M Goutallier (ISP),6M Goutallier (TM),Pre Goutallier (SSP) Missing flag,Pre Goutallier (SSC) Missing flag,Pre Goutallier (ISP) Missing flag,Pre Goutallier (TM) Missing flag,6M Goutallier (SSP) Missing flag,6M Goutallier (SSC) Missing flag,6M Goutallier (ISP) Missing flag,6M Goutallier (TM) Missing flag
0,70,2,1,-0.112260,-0.239010,0.529347,1.299314,1.236586,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,61,1,1,0.429514,-0.072467,-0.673177,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,73,2,1,-0.344449,-0.488825,-0.913682,-0.927981,-0.896158,1.0,0.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60,1,2,0.584307,0.268947,-0.071915,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75,2,2,-0.050343,-0.209865,0.529347,0.478732,0.450838,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7173,72,2,2,-0.371571,-1.414487,0.014274,-0.777214,-0.751792,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
7174,80,1,2,0.038328,0.032052,0.070864,-0.498668,-0.485071,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7175,83,2,1,0.778539,0.094077,-0.174047,-0.206960,-0.205745,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7176,65,2,1,0.048063,-0.588316,-1.114365,-1.123613,-1.083486,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
y_train

,POD 6M retear
0,0
1,0
2,0
3,0
4,0
...,...
7173,1
7174,1
7175,1
7176,1


In [5]:
columns = list(X_train.columns)
columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7',
 '0M ASES',
 '0M CSS',
 '0M ERabd',
 '0M ERside',
 '0M FF',
 '0M IR',
 '0M KSS',
 '0M MMTgrade',
 '0M MMTsec',
 '0M VAS(activity)',
 '0M VAS(resting)',
 '0M add',
 '2M ERabd',
 '2M ERside',
 '2M FF',
 '2M IR',
 '2M MMTgrade',
 '2M MMTsec',
 '2M add',
 '3M ASES',
 '3M CSS',
 '3M ERabd',
 '3M ERside',
 '3M FF',
 '3M IR',
 '3M KSS',
 '3M MMTgrade',
 '3M MMTsec',
 '3M VAS(activity)',
 '3M VAS(resting)',
 '3M add',
 '4M ASES',
 '4M CSS',
 '4M ERabd',
 '4M ERside',
 '4M FF',
 '4M IR',
 '4M KSS',
 '4M MMTgrade',
 '4M MMTsec',
 '4M VAS(activity)',
 '4M VAS(resting)',
 '4M add',
 '6M ASES',
 '6M CSS',
 

In [6]:
static_columns = columns[:25]

# static 데이터 칼럼
static_columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7']

In [7]:
seq_columns = columns[25:-16]

# 시퀀셜 데이터 관련 칼럼들
seq_columns_0M = seq_columns[:12]
seq_columns_2M = seq_columns[12:19]
seq_columns_3M = seq_columns[19:31]
seq_columns_4M = seq_columns[31:43]
seq_columns_6M = seq_columns[43:]

seq_columns_all = [seq_columns_0M, seq_columns_2M, seq_columns_3M, seq_columns_4M, seq_columns_6M]

for seq_col in seq_columns_all:
    print(seq_col)

['0M ASES', '0M CSS', '0M ERabd', '0M ERside', '0M FF', '0M IR', '0M KSS', '0M MMTgrade', '0M MMTsec', '0M VAS(activity)', '0M VAS(resting)', '0M add']
['2M ERabd', '2M ERside', '2M FF', '2M IR', '2M MMTgrade', '2M MMTsec', '2M add']
['3M ASES', '3M CSS', '3M ERabd', '3M ERside', '3M FF', '3M IR', '3M KSS', '3M MMTgrade', '3M MMTsec', '3M VAS(activity)', '3M VAS(resting)', '3M add']
['4M ASES', '4M CSS', '4M ERabd', '4M ERside', '4M FF', '4M IR', '4M KSS', '4M MMTgrade', '4M MMTsec', '4M VAS(activity)', '4M VAS(resting)', '4M add']
['6M ASES', '6M CSS', '6M ERabd', '6M ERside', '6M FF', '6M IR', '6M KSS', '6M MMTgrade', '6M MMTsec', '6M VAS(activity)', '6M VAS(resting)', '6M add']


In [8]:
goutallier_columns = columns[-16:]

# goutaliar 관련 칼럼들
goutallier_columns_0M = goutallier_columns [:4]
goutallier_columns_6M = goutallier_columns [4:8]
goutallier_columns_0M_missing = goutallier_columns [8:12]
goutallier_columns_6M_missing = goutallier_columns [12:]

print(goutallier_columns_0M)
print(goutallier_columns_6M)
print(goutallier_columns_0M_missing)
print(goutallier_columns_6M_missing)

['Pre Goutallier (SSP)', 'Pre Goutallier (SSC)', 'Pre Goutallier (ISP)', 'Pre Goutallier (TM)']
['6M Goutallier (SSP)', '6M Goutallier (SSC)', '6M Goutallier (ISP)', '6M Goutallier (TM)']
['Pre Goutallier (SSP) Missing flag', 'Pre Goutallier (SSC) Missing flag', 'Pre Goutallier (ISP) Missing flag', 'Pre Goutallier (TM) Missing flag']
['6M Goutallier (SSP) Missing flag', '6M Goutallier (SSC) Missing flag', '6M Goutallier (ISP) Missing flag', '6M Goutallier (TM) Missing flag']


In [9]:
len(columns) == len(static_columns) + len(seq_columns) + len(goutallier_columns)

True

In [10]:
label_column = "POD 6M retear"
output_columns = ["6M ASES", "6M CSS", "6M KSS", "6M VAS(activity)", "6M VAS(resting)"]
input_columns = static_columns + [column for column in seq_columns if column not in output_columns] + goutallier_columns

In [11]:
output_columns

['6M ASES', '6M CSS', '6M KSS', '6M VAS(activity)', '6M VAS(resting)']

In [12]:
input_columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7',
 '0M ASES',
 '0M CSS',
 '0M ERabd',
 '0M ERside',
 '0M FF',
 '0M IR',
 '0M KSS',
 '0M MMTgrade',
 '0M MMTsec',
 '0M VAS(activity)',
 '0M VAS(resting)',
 '0M add',
 '2M ERabd',
 '2M ERside',
 '2M FF',
 '2M IR',
 '2M MMTgrade',
 '2M MMTsec',
 '2M add',
 '3M ASES',
 '3M CSS',
 '3M ERabd',
 '3M ERside',
 '3M FF',
 '3M IR',
 '3M KSS',
 '3M MMTgrade',
 '3M MMTsec',
 '3M VAS(activity)',
 '3M VAS(resting)',
 '3M add',
 '4M ASES',
 '4M CSS',
 '4M ERabd',
 '4M ERside',
 '4M FF',
 '4M IR',
 '4M KSS',
 '4M MMTgrade',
 '4M MMTsec',
 '4M VAS(activity)',
 '4M VAS(resting)',
 '4M add',
 '6M ERabd',
 '6M ERside

In [13]:
X_train[input_columns]

,나이,"성별 (M:1,F:2)","Rt:1,Lt:2",Height,Weight,"Tearsize (AP,cm)",Tearsize (ML),Tearsize (retraction),"흡연여부 (비흡연:1,흡연:2)","흡연여부 (비흡연:1,흡연:2) Missing flag",...,6M Goutallier (ISP),6M Goutallier (TM),Pre Goutallier (SSP) Missing flag,Pre Goutallier (SSC) Missing flag,Pre Goutallier (ISP) Missing flag,Pre Goutallier (TM) Missing flag,6M Goutallier (SSP) Missing flag,6M Goutallier (SSC) Missing flag,6M Goutallier (ISP) Missing flag,6M Goutallier (TM) Missing flag
0,70,2,1,-0.112260,-0.239010,0.529347,1.299314,1.236586,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,61,1,1,0.429514,-0.072467,-0.673177,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,73,2,1,-0.344449,-0.488825,-0.913682,-0.927981,-0.896158,1.0,0.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60,1,2,0.584307,0.268947,-0.071915,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75,2,2,-0.050343,-0.209865,0.529347,0.478732,0.450838,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7173,72,2,2,-0.371571,-1.414487,0.014274,-0.777214,-0.751792,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
7174,80,1,2,0.038328,0.032052,0.070864,-0.498668,-0.485071,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7175,83,2,1,0.778539,0.094077,-0.174047,-0.206960,-0.205745,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7176,65,2,1,0.048063,-0.588316,-1.114365,-1.123613,-1.083486,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
pd.concat([y_train, X_train[output_columns]], axis=1)

,POD 6M retear,6M ASES,6M CSS,6M KSS,6M VAS(activity),6M VAS(resting)
0,0,1.377047,1.448052,0.201114,1.201822,1.202527
1,0,0.756417,0.258163,1.336235,-1.788307,-1.761167
2,0,-0.655014,1.087954,-1.004221,1.201822,1.202527
3,0,-0.594953,-0.054966,-0.699962,1.201822,1.202527
4,0,0.566225,-1.526670,-0.126550,0.205112,0.214629
...,...,...,...,...,...,...
7173,1,0.576435,-0.284644,0.671297,0.560877,0.567248
7174,1,-0.788472,0.328121,-0.568735,0.205112,0.214629
7175,1,0.678394,-0.617836,-0.355947,2.198532,2.190425
7176,1,0.932377,0.515679,0.898310,0.205112,0.214629


In [15]:
def get_dataset(split):
  assert split in ["train", "val", "test"]
  
  X_file_name = f"{DATA_PATH}/X_{split}.csv"
  y_file_name = f"{DATA_PATH}/y_{split}.csv"

  X = pd.read_csv(X_file_name)
  y = pd.read_csv(y_file_name)
  
  # static 데이터
  X_static_tensor = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)

  # 시기별 sequential 데이터
  X_seq_tensor_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_4M = torch.tensor(X[seq_columns_4M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_6M = torch.tensor(X[seq_columns_6M].to_numpy(), dtype=torch.float32)
  
  #0M, 6M goutalier 데이터
  X_goutalier_tensor_0M = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
  X_goutalier_tensor_6M = torch.tensor(X[goutallier_columns_6M + goutallier_columns_6M_missing].to_numpy(), dtype=torch.float32)
  
  # 전체 인풋 데이터
  X_tensor = torch.tensor(X[input_columns].to_numpy(), dtype=torch.float32)
  
  # 6M 예측 데이터
  y_tensor = torch.tensor(pd.concat([y, X[output_columns]], axis=1).to_numpy(), dtype=torch.float32)

  return TensorDataset(X_tensor, X_static_tensor, X_seq_tensor_0M, X_seq_tensor_2M, X_seq_tensor_3M, X_seq_tensor_4M, X_seq_tensor_6M, X_goutalier_tensor_0M, X_goutalier_tensor_6M, y_tensor)

In [16]:
trainset = get_dataset("train")
valset = get_dataset("val")
testset = get_dataset("test")

print("기존 데이터셋 구조 확인")
print(f"Trainset size: {len(trainset)}")
print(f"Validset size: {len(valset)}") # jiseock
print(f"Testset size: {len(testset)}")

기존 데이터셋 구조 확인
Trainset size: 7178
Validset size: 647
Testset size: 100


In [17]:
# 특징 크기 확인
static_features = len(static_columns)
seq_features_0M = len(seq_columns_0M)
seq_features_2M = len(seq_columns_2M)
seq_features_3M = len(seq_columns_3M)
seq_features_4M = len(seq_columns_4M)
seq_features_6M = len(seq_columns_6M)
goutallier_features_0M = len(goutallier_columns_0M) + len(goutallier_columns_0M_missing)
goutallier_features_6M = len(goutallier_columns_6M) + len(goutallier_columns_6M_missing)

print(f"Static features: {static_features}")
print(f"0M features: {seq_features_0M}, 2M features: {seq_features_2M}, 3M features: {seq_features_3M}")
print(f"4M features: {seq_features_4M}, 6M features: {seq_features_6M}")
print(f"0M Goutallier features: {goutallier_features_0M}, 6M Goutallier features: {goutallier_features_6M}")

Static features: 25
0M features: 12, 2M features: 7, 3M features: 12
4M features: 12, 6M features: 12
0M Goutallier features: 8, 6M Goutallier features: 8


In [18]:
# 각 모델별 데이터셋 생성 함수
def get_dataset_model1(split):
    """Model 1: static + 0M + 0M_goutallier → 2M"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)
    
    # 입력: static + 0M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 2M
    y_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_0M_goutallier, y_2M)

def get_dataset_model2(split):
    """Model 2: static + 0M + 2M + 0M_goutallier → 3M"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)
    
    # 입력: static + 0M + 2M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 3M
    y_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_2M, X_0M_goutallier, y_3M)

def get_dataset_model3(split):
    """Model 3: static + 0M + 2M + 3M + 0M_goutallier → 4M"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)
    
    # 입력: static + 0M + 2M + 3M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    X_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 4M
    y_4M = torch.tensor(X[seq_columns_4M].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_2M, X_3M, X_0M_goutallier, y_4M)

def get_dataset_model4(split):
    """Model 4: static + 0M + 2M + 3M + 4M + 0M_goutallier → 6M + y + 6M_goutallier"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)
    
    # 입력: static + 0M + 2M + 3M + 4M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    X_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
    X_4M = torch.tensor(X[seq_columns_4M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 6M + y + 6M_goutallier
    y_6M = torch.tensor(X[seq_columns_6M].to_numpy(), dtype=torch.float32)
    y_label = torch.tensor(y[label_column].to_numpy(), dtype=torch.float32).unsqueeze(1)
    y_6M_goutallier = torch.tensor(X[goutallier_columns_6M + goutallier_columns_6M_missing].to_numpy(), dtype=torch.float32)
    
    # 결합: [6M features (12) + y (1) + 6M_goutallier (8)] = 21
    y_combined = torch.cat([y_6M, y_label, y_6M_goutallier], dim=1)
    
    return TensorDataset(X_static, X_0M, X_2M, X_3M, X_4M, X_0M_goutallier, y_combined)

# 데이터셋 생성
trainset_model1 = get_dataset_model1("train")
testset_model1 = get_dataset_model1("test")
valset_model1 = get_dataset_model1("val")

trainset_model2 = get_dataset_model2("train")
testset_model2 = get_dataset_model2("test")
valset_model2 = get_dataset_model2("val")

trainset_model3 = get_dataset_model3("train")
testset_model3 = get_dataset_model3("test")
valset_model3 = get_dataset_model3("val")

trainset_model4 = get_dataset_model4("train")
testset_model4 = get_dataset_model4("test")
valset_model4 = get_dataset_model4("val")

print(f"Model 1 - Train: {len(trainset_model1)}, Test: {len(testset_model1)}, Valid: {len(valset_model1)}")
print(f"Model 2 - Train: {len(trainset_model2)}, Test: {len(testset_model2)}, Valid: {len(valset_model2)}")
print(f"Model 3 - Train: {len(trainset_model3)}, Test: {len(testset_model3)}, Valid: {len(valset_model3)}")
print(f"Model 4 - Train: {len(trainset_model4)}, Test: {len(testset_model4)}, Valid: {len(valset_model4)}")


Model 1 - Train: 7178, Test: 100, Valid: 647
Model 2 - Train: 7178, Test: 100, Valid: 647
Model 3 - Train: 7178, Test: 100, Valid: 647
Model 4 - Train: 7178, Test: 100, Valid: 647


In [ ]:
class LossHistoryCallback(Callback):
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.test_losses = []
    
    def on_train_epoch_end(self, trainer, pl_module):
        if len(self.train_losses) == 0:
            print(f"[Train] Available metrics: {list(trainer.callback_metrics.keys())}")
        
        train_loss = trainer.callback_metrics.get('train/loss_epoch')
        if train_loss is not None:
            self.train_losses.append(train_loss.item())
        else:
            print(f"Warning: train/loss_epoch not found!")
    
    def on_validation_epoch_end(self, trainer, pl_module):
        if len(self.test_losses) == 0:
            print(f"[Val] Available metrics: {list(trainer.callback_metrics.keys())}")
        
        val_loss = trainer.callback_metrics.get('val/loss')
        if val_loss is not None:
            self.test_losses.append(val_loss.item())
        else:
            print(f"Warning: val/loss not found!")

class CleanupCheckpointCallback(Callback):
    """학습 종료 후 최고 성능 체크포인트만 남기고 나머지 제거"""
    def __init__(self, checkpoint_dir, filename_prefix):
        super().__init__()
        self.checkpoint_dir = checkpoint_dir
        self.filename_prefix = filename_prefix
    
    def on_train_end(self, trainer, pl_module):
        import os
        import glob
        
        # 체크포인트 디렉토리의 모든 파일 확인
        checkpoint_files = glob.glob(os.path.join(self.checkpoint_dir, f"{self.filename_prefix}*.ckpt"))
        
        if len(checkpoint_files) == 0:
            return
        
        # 최고 성능 체크포인트 찾기 (ModelCheckpoint가 저장한 best 체크포인트)
        best_checkpoint = None
        for callback in trainer.callbacks:
            if isinstance(callback, ModelCheckpoint):
                best_checkpoint = callback.best_model_path
                break
        
        # 최고 성능 체크포인트가 있으면 나머지 제거
        if best_checkpoint and os.path.exists(best_checkpoint):
            removed_count = 0
            for checkpoint_file in checkpoint_files:
                if checkpoint_file != best_checkpoint:
                    try:
                        os.remove(checkpoint_file)
                        removed_count += 1
                    except Exception as e:
                        print(f"Warning: 체크포인트 삭제 실패 {checkpoint_file}: {e}")
            
            if removed_count > 0:
                print(f"[체크포인트 정리] {removed_count}개의 체크포인트 제거 완료. 최고 성능 체크포인트만 유지: {os.path.basename(best_checkpoint)}")
        else:
            # best 체크포인트를 찾지 못한 경우, 파일명으로 최신 것만 남기기
            if checkpoint_files:
                # 파일 수정 시간 기준으로 정렬
                checkpoint_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
                best_checkpoint = checkpoint_files[0]
                
                removed_count = 0
                for checkpoint_file in checkpoint_files[1:]:
                    try:
                        os.remove(checkpoint_file)
                        removed_count += 1
                    except Exception as e:
                        print(f"Warning: 체크포인트 삭제 실패 {checkpoint_file}: {e}")
                
                if removed_count > 0:
                    print(f"[체크포인트 정리] {removed_count}개의 체크포인트 제거 완료. 최신 체크포인트만 유지: {os.path.basename(best_checkpoint)}")


### Optuna 모델 최적화

In [20]:
# ============================================================================
# NAS 기반 Optuna 하이퍼파라미터 최적화
# ============================================================================

def create_encoder(trial, input_dim, name_prefix, min_units=32, max_units=256, min_layers=1, max_layers=3):
    """동적으로 인코더 생성 - NAS 기반"""
    n_layers = trial.suggest_int(f'{name_prefix}_n_layers', min_layers, max_layers)
    layers = []
    prev_dim = input_dim
    
    for i in range(n_layers):
        units = trial.suggest_int(f'{name_prefix}_units_{i}', min_units, max_units, step=32)
        dropout = trial.suggest_float(f'{name_prefix}_dropout_{i}', 0.0, 0.5, step=0.05)
        use_batch_norm = trial.suggest_categorical(f'{name_prefix}_batch_norm_{i}', [True, False])
        
        layers.append(nn.Linear(prev_dim, units))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(units))
        else:
            layers.append(nn.LayerNorm(units))
        layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        prev_dim = units
    
    return nn.Sequential(*layers), prev_dim

def create_output_head(trial, input_dim, output_dim, name_prefix, min_units=64, max_units=512, min_layers=1, max_layers=4):
    """동적으로 출력 헤드 생성 - NAS 기반"""
    n_layers = trial.suggest_int(f'{name_prefix}_n_layers', min_layers, max_layers)
    layers = []
    prev_dim = input_dim
    
    for i in range(n_layers - 1):
        units = trial.suggest_int(f'{name_prefix}_units_{i}', min_units, max_units, step=32)
        dropout = trial.suggest_float(f'{name_prefix}_dropout_{i}', 0.0, 0.5, step=0.05)
        use_batch_norm = trial.suggest_categorical(f'{name_prefix}_batch_norm_{i}', [True, False])
        
        layers.append(nn.Linear(prev_dim, units))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(units))
        else:
            layers.append(nn.LayerNorm(units))
        layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        prev_dim = units
    
    # 마지막 레이어 (출력)
    layers.append(nn.Linear(prev_dim, output_dim))
    
    return nn.Sequential(*layers)

print("NAS 기반 인코더/헤드 생성 함수 정의 완료")


NAS 기반 인코더/헤드 생성 함수 정의 완료


In [21]:
# Model 1 최적화 버전
class OptimizedSequentialMLP1(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, goutallier_0M_features, out_features_2M):
        super().__init__()
        
        # 인코더 생성
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=64, max_units=256
        )
        
        self.seq_0M_encoder, seq_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        # 특징 결합 후 출력 헤드
        feat_dim = static_out + seq_out + goutalier_out
        self.output_head = create_output_head(
            trial, feat_dim, out_features_2M, 'output_head', min_units=128, max_units=512
        )
        
        # 학습 파라미터
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, goutallier_0M_feat], dim=1)
        output = self.output_head(combined)
        return output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_2M = batch
        pred_2M = self.forward(x_static, x_0M, x_0M_goutallier)
        loss = F.mse_loss(pred_2M, y_2M)
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.train_mse.update(pred_2M, y_2M)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_2M = batch
        pred_2M = self.forward(x_static, x_0M, x_0M_goutallier)
        loss = F.mse_loss(pred_2M, y_2M)
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.val_mse.update(pred_2M, y_2M)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/mse", self.train_mse.compute())
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/mse", self.val_mse.compute())
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_2M = batch
        pred_2M = self.forward(x_static, x_0M, x_0M_goutallier)
        loss = F.mse_loss(pred_2M, y_2M)
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.test_mse.update(pred_2M, y_2M)
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/mse", self.test_mse.compute())
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/loss"
            }
        }

# Model 2 최적화 버전
class OptimizedSequentialMLP2(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, seq_2M_features, goutallier_0M_features, out_features_3M):
        super().__init__()
        
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=32, max_units=128
        )
        
        self.seq_0M_encoder, seq_0M_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.seq_2M_encoder, seq_2M_out = create_encoder(
            trial, seq_2M_features, 'seq_2M_encoder', min_units=32, max_units=128
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        feat_dim = static_out + seq_0M_out + seq_2M_out + goutalier_out
        self.output_head = create_output_head(
            trial, feat_dim, out_features_3M, 'output_head', min_units=64, max_units=256
        )
        
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_2M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        seq_2M_feat = self.seq_2M_encoder(x_2M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, seq_2M_feat, goutallier_0M_feat], dim=1)
        output = self.output_head(combined)
        return output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
        pred_3M = self.forward(x_static, x_0M, x_2M, x_0M_goutallier)
        loss = F.mse_loss(pred_3M, y_3M)
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.train_mse.update(pred_3M, y_3M)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
        pred_3M = self.forward(x_static, x_0M, x_2M, x_0M_goutallier)
        loss = F.mse_loss(pred_3M, y_3M)
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.val_mse.update(pred_3M, y_3M)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/mse", self.train_mse.compute())
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/mse", self.val_mse.compute())
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
        pred_3M = self.forward(x_static, x_0M, x_2M, x_0M_goutallier)
        loss = F.mse_loss(pred_3M, y_3M)
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.test_mse.update(pred_3M, y_3M)
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/mse", self.test_mse.compute())
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/loss"
            }
        }

# Model 3 최적화 버전
class OptimizedSequentialMLP3(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, seq_2M_features, seq_3M_features, goutallier_0M_features, out_features_4M):
        super().__init__()
        
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=32, max_units=128
        )
        
        self.seq_0M_encoder, seq_0M_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.seq_2M_encoder, seq_2M_out = create_encoder(
            trial, seq_2M_features, 'seq_2M_encoder', min_units=32, max_units=128
        )
        
        self.seq_3M_encoder, seq_3M_out = create_encoder(
            trial, seq_3M_features, 'seq_3M_encoder', min_units=64, max_units=256
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        feat_dim = static_out + seq_0M_out + seq_2M_out + seq_3M_out + goutalier_out
        self.output_head = create_output_head(
            trial, feat_dim, out_features_4M, 'output_head', min_units=64, max_units=256
        )
        
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_2M, x_3M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        seq_2M_feat = self.seq_2M_encoder(x_2M)
        seq_3M_feat = self.seq_3M_encoder(x_3M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, seq_2M_feat, seq_3M_feat, goutallier_0M_feat], dim=1)
        output = self.output_head(combined)
        return output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
        pred_4M = self.forward(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
        loss = F.mse_loss(pred_4M, y_4M)
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.train_mse.update(pred_4M, y_4M)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
        pred_4M = self.forward(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
        loss = F.mse_loss(pred_4M, y_4M)
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.val_mse.update(pred_4M, y_4M)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/mse", self.train_mse.compute())
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/mse", self.val_mse.compute())
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
        pred_4M = self.forward(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
        loss = F.mse_loss(pred_4M, y_4M)
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.test_mse.update(pred_4M, y_4M)
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/mse", self.test_mse.compute())
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/loss"
            }
        }

# Model 4 최적화 버전 (분류 + 회귀)
class OptimizedSequentialMLP4(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, seq_2M_features, seq_3M_features, seq_4M_features, goutallier_0M_features, out_features_total):
        super().__init__()
        self.register_buffer('pos_weight', torch.tensor([1.0]))
        
        # 회귀 손실 가중치 최적화
        self.reg_loss_weight = trial.suggest_float('reg_loss_weight', 0.1, 0.5, step=0.05)
        
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=32, max_units=128
        )
        
        self.seq_0M_encoder, seq_0M_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.seq_2M_encoder, seq_2M_out = create_encoder(
            trial, seq_2M_features, 'seq_2M_encoder', min_units=32, max_units=128
        )
        
        self.seq_3M_encoder, seq_3M_out = create_encoder(
            trial, seq_3M_features, 'seq_3M_encoder', min_units=64, max_units=256
        )
        
        self.seq_4M_encoder, seq_4M_out = create_encoder(
            trial, seq_4M_features, 'seq_4M_encoder', min_units=64, max_units=256
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        feat_dim = static_out + seq_0M_out + seq_2M_out + seq_3M_out + seq_4M_out + goutalier_out
        
        # 분류 헤드
        self.clshead = create_output_head(
            trial, feat_dim, 1, 'clshead', min_units=64, max_units=256
        )
        
        # 회귀 헤드
        self.reghead = create_output_head(
            trial, feat_dim, out_features_total - 1, 'reghead', min_units=128, max_units=512
        )
        
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_roc = BinaryAUROC()
        self.val_roc = BinaryAUROC()
        self.test_roc = BinaryAUROC()
        self.val_ap = BinaryAveragePrecision()
        self.test_ap = BinaryAveragePrecision()
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        seq_2M_feat = self.seq_2M_encoder(x_2M)
        seq_3M_feat = self.seq_3M_encoder(x_3M)
        seq_4M_feat = self.seq_4M_encoder(x_4M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, seq_2M_feat, seq_3M_feat, seq_4M_feat, goutallier_0M_feat], dim=1)
        
        logits = self.clshead(combined)
        regs = self.reghead(combined)
        
        output = torch.cat([logits, regs], dim=1)
        return logits, regs, output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
        y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
        y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
        
        logits, regs, _ = self.forward(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
        
        clf_loss = F.binary_cross_entropy_with_logits(logits, y_label, pos_weight=self.pos_weight)
        reg_loss = F.smooth_l1_loss(regs, y_reg)
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.log("train/clf_loss", clf_loss)
        self.log("train/reg_loss", reg_loss)
        
        probs = logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.train_roc.update(probs, targets)
        self.train_mse.update(regs, y_reg)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
        y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
        y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
        
        logits, regs, _ = self.forward(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
        
        clf_loss = F.binary_cross_entropy_with_logits(logits, y_label, pos_weight=self.pos_weight)
        reg_loss = F.smooth_l1_loss(regs, y_reg)
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.log("val/clf_loss", clf_loss)
        self.log("val/reg_loss", reg_loss)
        
        probs = logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.val_roc.update(probs, targets)
        self.val_ap.update(probs, targets)
        self.val_mse.update(regs, y_reg)
        
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/roc", self.train_roc.compute())
        self.log("train/mse", self.train_mse.compute())
        self.train_roc.reset()
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/roc", self.val_roc.compute())
        self.log("val/ap", self.val_ap.compute())
        self.log("val/mse", self.val_mse.compute())
        self.val_roc.reset()
        self.val_ap.reset()
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
        y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
        y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
        
        logits, regs, _ = self.forward(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
        
        clf_loss = F.binary_cross_entropy_with_logits(logits, y_label, pos_weight=self.pos_weight)
        reg_loss = F.smooth_l1_loss(regs, y_reg)
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.log("test/clf_loss", clf_loss)
        self.log("test/reg_loss", reg_loss)
        
        probs = logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.test_roc.update(probs, targets)
        self.test_ap.update(probs, targets)
        self.test_mse.update(regs, y_reg)
        
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/roc", self.test_roc.compute())
        self.log("test/ap", self.test_ap.compute())
        self.log("test/mse", self.test_mse.compute())
        self.test_roc.reset()
        self.test_ap.reset()
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/roc"
            }
        }

print("최적화된 모델 클래스 정의 완료")


최적화된 모델 클래스 정의 완료


In [22]:
# 연결된 시계열 모델 클래스 (4개 모델을 순차적으로 실행)
class SequentialModel(nn.Module):
    """0M 입력만으로 6M 예측하는 연결된 시계열 모델"""
    def __init__(self, model1, model2, model3, model4):
        super().__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        
        # 평가 모드로 설정
        self.model1.eval()
        self.model2.eval()
        self.model3.eval()
        self.model4.eval()
    
    @torch.no_grad()
    def forward(self, x_static, x_0M, x_0M_goutallier):
        """
        입력: static, 0M, 0M_goutallier
        출력: 6M features, y (logits), 6M_goutallier
        """
        # Model 1: static + 0M + 0M_goutallier → 2M
        pred_2M = self.model1(x_static, x_0M, x_0M_goutallier)
        
        # Model 2: static + 0M + 2M + 0M_goutallier → 3M
        pred_3M = self.model2(x_static, x_0M, pred_2M, x_0M_goutallier)
        
        # Model 3: static + 0M + 2M + 3M + 0M_goutallier → 4M
        pred_4M = self.model3(x_static, x_0M, pred_2M, pred_3M, x_0M_goutallier)
        
        # Model 4: static + 0M + 2M + 3M + 4M + 0M_goutallier → 6M + y + 6M_goutallier
        logits, regs, output = self.model4(x_static, x_0M, pred_2M, pred_3M, pred_4M, x_0M_goutallier)
        
        # 출력 분리: [6M (12) + y (1) + 6M_goutallier (8)]
        pred_6M = regs[:, :seq_features_6M]
        pred_y_logits = logits
        pred_6M_goutallier = regs[:, seq_features_6M:]
        
        return {
            'pred_2M': pred_2M,
            'pred_3M': pred_3M,
            'pred_4M': pred_4M,
            'pred_6M': pred_6M,
            'pred_y_logits': pred_y_logits,
            'pred_6M_goutallier': pred_6M_goutallier,
            'output': output
        }

In [ ]:
# ============================================================================
# Optuna 최적화 함수들
# ============================================================================
CHECKPOINT_DIR = "../checkpoint"

def optimize_model1(trial):
    """Model 1 최적화"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP1(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_2M=seq_features_2M
    )
    
    trainloader = DataLoader(trainset_model1, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model1, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=50,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/loss"),
            ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='optuna-model1-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model1-best'),
            EarlyStopping(monitor='val/loss', mode='min', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    return trainer.callback_metrics["val/loss"].item()

def optimize_model2(trial):
    """Model 2 최적화"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP2(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        seq_2M_features=seq_features_2M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_3M=seq_features_3M
    )
    
    trainloader = DataLoader(trainset_model2, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model2, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=50,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/loss"),
            ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='optuna-model2-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model2-best'),
            EarlyStopping(monitor='val/loss', mode='min', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    return trainer.callback_metrics["val/loss"].item()

def optimize_model3(trial):
    """Model 3 최적화"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP3(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        seq_2M_features=seq_features_2M,
        seq_3M_features=seq_features_3M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_4M=seq_features_4M
    )
    
    trainloader = DataLoader(trainset_model3, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model3, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=50,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/loss"),
            ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='optuna-model3-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model3-best'),
            EarlyStopping(monitor='val/loss', mode='min', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    return trainer.callback_metrics["val/loss"].item()

def optimize_model4(trial):
    """Model 4 최적화 (ROC AUC 최대화)"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP4(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        seq_2M_features=seq_features_2M,
        seq_3M_features=seq_features_3M,
        seq_4M_features=seq_features_4M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_total=seq_features_6M + 1 + goutallier_features_6M
    )
    
    trainloader = DataLoader(trainset_model4, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model4, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=50,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/roc"),
            ModelCheckpoint(monitor='val/roc', mode='max', save_top_k=1, save_last=False, filename='optuna-model4-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model4-best'),
            EarlyStopping(monitor='val/roc', mode='max', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    # ROC AUC를 최대화 (음수로 반환하여 최소화 문제로 변환)
    return -trainer.callback_metrics["val/roc"].item()

print("Optuna 최적화 함수 정의 완료")


Optuna 최적화 함수 정의 완료


### Optuna 실행 및 파라미터 최적화

In [ ]:
n_trials = 20  # 각 모델당 시도 횟수 (필요에 따라 조정 가능)

print("=" * 80)
print("NAS 기반 Optuna 하이퍼파라미터 최적화 시작")
print("=" * 80)

# Model 1 최적화
print("\n[Model 1] 최적화 시작...")
study1 = optuna.create_study(
    direction='minimize',
    study_name='sequential_mlp1_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study1.optimize(optimize_model1, n_trials=n_trials, show_progress_bar=True)

print(f"\n[Model 1] 최적화 완료!")
print(f"최고 성능: {study1.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study1.best_params.items():
    print(f"  {key}: {value}")

# Model 2 최적화
print("\n[Model 2] 최적화 시작...")
study2 = optuna.create_study(
    direction='minimize',
    study_name='sequential_mlp2_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study2.optimize(optimize_model2, n_trials=n_trials, show_progress_bar=True)

print(f"\n[Model 2] 최적화 완료!")
print(f"최고 성능: {study2.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study2.best_params.items():
    print(f"  {key}: {value}")

# Model 3 최적화
print("\n[Model 3] 최적화 시작...")
study3 = optuna.create_study(
    direction='minimize',
    study_name='sequential_mlp3_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study3.optimize(optimize_model3, n_trials=n_trials, show_progress_bar=True)

print(f"\n[Model 3] 최적화 완료!")
print(f"최고 성능: {study3.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study3.best_params.items():
    print(f"  {key}: {value}")

# Model 4 최적화
print("\n[Model 4] 최적화 시작...")
study4 = optuna.create_study(
    direction='minimize',  # 음수 ROC AUC를 최소화 = ROC AUC 최대화
    study_name='sequential_mlp4_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study4.optimize(optimize_model4, n_trials=n_trials, show_progress_bar=True)

print(f"\n[Model 4] 최적화 완료!")
print(f"최고 성능 (음수 ROC AUC): {study4.best_value:.6f}")
print(f"실제 최고 ROC AUC: {-study4.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study4.best_params.items():
    print(f"  {key}: {value}")

print("\n" + "=" * 80)
print("모든 모델 최적화 완료!")
print("=" * 80)

[I 2025-11-09 12:04:04,784] A new study created in memory with name: sequential_mlp1_optimization


NAS 기반 Optuna 하이퍼파라미터 최적화 시작

[Model 1] 최적화 시작...


  0%|          | 0/100 [00:00<?, ?it/s]Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
/data/miniconda3/envs/arcr/lib/python3.10/site-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Co

[I 2025-11-09 12:04:34,781] Trial 0 finished with value: 1.0964479446411133 and parameters: {'batch_size': 128, 'static_encoder_n_layers': 2, 'static_encoder_units_0': 64, 'static_encoder_dropout_0': 0.0, 'static_encoder_batch_norm_0': True, 'static_encoder_units_1': 96, 'static_encoder_dropout_1': 0.15000000000000002, 'static_encoder_batch_norm_1': True, 'seq_0M_encoder_n_layers': 2, 'seq_0M_encoder_units_0': 256, 'seq_0M_encoder_dropout_0': 0.5, 'seq_0M_encoder_batch_norm_0': True, 'seq_0M_encoder_units_1': 64, 'seq_0M_encoder_dropout_1': 0.15000000000000002, 'seq_0M_encoder_batch_norm_1': True, 'goutallier_0M_encoder_n_layers': 3, 'goutallier_0M_encoder_units_0': 64, 'goutallier_0M_encoder_dropout_0': 0.25, 'goutallier_0M_encoder_batch_norm_0': False, 'goutallier_0M_encoder_units_1': 32, 'goutallier_0M_encoder_dropout_1': 0.2, 'goutallier_0M_encoder_batch_norm_1': False, 'goutallier_0M_encoder_units_2': 96, 'goutallier_0M_encoder_dropout_2': 0.1, 'goutallier_0M_encoder_batch_norm_2'

Best trial: 1. Best value: 1.06607:   2%|▏         | 2/100 [00:58<47:09, 28.87s/it]Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name                  | Type             | Params | Mode 
-------------------------------------------------------------------
0 | static_encoder        | Sequential       | 47.1 K | train
1 | seq_0M_encoder        | Sequential       | 91.0 K | train
2 | goutallier_0M_encoder | Sequential       | 1.4 K  | train
3 | output_head           | Sequential       | 427 K  | train
4 | train_mse             | MeanSquaredError | 0      | train
5 | val_mse               | MeanSquaredError | 0      | trai

...

[I 2025-11-09 16:51:01,881] Trial 97 finished with value: -0.7733559608459473 and parameters: {'batch_size': 64, 'reg_loss_weight': 0.30000000000000004, 'static_encoder_n_layers': 3, 'static_encoder_units_0': 128, 'static_encoder_dropout_0': 0.5, 'static_encoder_batch_norm_0': False, 'static_encoder_units_1': 64, 'static_encoder_dropout_1': 0.5, 'static_encoder_batch_norm_1': False, 'static_encoder_units_2': 128, 'static_encoder_dropout_2': 0.5, 'static_encoder_batch_norm_2': True, 'seq_0M_encoder_n_layers': 2, 'seq_0M_encoder_units_0': 192, 'seq_0M_encoder_dropout_0': 0.5, 'seq_0M_encoder_batch_norm_0': False, 'seq_0M_encoder_units_1': 160, 'seq_0M_encoder_dropout_1': 0.1, 'seq_0M_encoder_batch_norm_1': False, 'seq_2M_encoder_n_layers': 3, 'seq_2M_encoder_units_0': 64, 'seq_2M_encoder_dropout_0': 0.30000000000000004, 'seq_2M_encoder_batch_norm_0': True, 'seq_2M_encoder_units_1': 128, 'seq_2M_encoder_dropout_1': 0.45, 'seq_2M_encoder_batch_norm_1': False, 'seq_2M_encoder_units_2': 32, 

Best trial: 54. Best value: -0.839481:  99%|█████████▉| 99/100 [2:03:52<01:56, 116.81s/it]Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

   | Name                  | Type                   | Params | Mode 
--------------------------------------------------------------------------
0  | static_encoder        | Sequential             | 16.3 K | train
1  | seq_0M_encoder        | Sequential             | 27.1 K | train
2  | seq_2M_encoder        | Sequential             | 16.6 K | train
3  | seq_3M_encoder        | Sequential             | 60.9 K | train
4  | seq_4M_encoder        | Sequential             | 61.5 K | train
5  |

[I 2025-11-09 16:52:52,130] Trial 98 finished with value: -0.7951953411102295 and parameters: {'batch_size': 32, 'reg_loss_weight': 0.2, 'static_encoder_n_layers': 3, 'static_encoder_units_0': 128, 'static_encoder_dropout_0': 0.35000000000000003, 'static_encoder_batch_norm_0': False, 'static_encoder_units_1': 64, 'static_encoder_dropout_1': 0.15000000000000002, 'static_encoder_batch_norm_1': False, 'static_encoder_units_2': 32, 'static_encoder_dropout_2': 0.0, 'static_encoder_batch_norm_2': True, 'seq_0M_encoder_n_layers': 1, 'seq_0M_encoder_units_0': 160, 'seq_0M_encoder_dropout_0': 0.45, 'seq_0M_encoder_batch_norm_0': False, 'seq_2M_encoder_n_layers': 3, 'seq_2M_encoder_units_0': 64, 'seq_2M_encoder_dropout_0': 0.45, 'seq_2M_encoder_batch_norm_0': True, 'seq_2M_encoder_units_1': 96, 'seq_2M_encoder_dropout_1': 0.4, 'seq_2M_encoder_batch_norm_1': True, 'seq_2M_encoder_units_2': 96, 'seq_2M_encoder_dropout_2': 0.30000000000000004, 'seq_2M_encoder_batch_norm_2': True, 'seq_3M_encoder_n_

Best trial: 54. Best value: -0.839481: 100%|██████████| 100/100 [2:05:08<00:00, 75.08s/it] 

[I 2025-11-09 16:54:07,881] Trial 99 finished with value: -0.7465420961380005 and parameters: {'batch_size': 64, 'reg_loss_weight': 0.2, 'static_encoder_n_layers': 3, 'static_encoder_units_0': 128, 'static_encoder_dropout_0': 0.35000000000000003, 'static_encoder_batch_norm_0': True, 'static_encoder_units_1': 64, 'static_encoder_dropout_1': 0.15000000000000002, 'static_encoder_batch_norm_1': False, 'static_encoder_units_2': 64, 'static_encoder_dropout_2': 0.0, 'static_encoder_batch_norm_2': True, 'seq_0M_encoder_n_layers': 2, 'seq_0M_encoder_units_0': 128, 'seq_0M_encoder_dropout_0': 0.45, 'seq_0M_encoder_batch_norm_0': False, 'seq_0M_encoder_units_1': 192, 'seq_0M_encoder_dropout_1': 0.0, 'seq_0M_encoder_batch_norm_1': True, 'seq_2M_encoder_n_layers': 3, 'seq_2M_encoder_units_0': 64, 'seq_2M_encoder_dropout_0': 0.5, 'seq_2M_encoder_batch_norm_0': True, 'seq_2M_encoder_units_1': 96, 'seq_2M_encoder_dropout_1': 0.4, 'seq_2M_encoder_batch_norm_1': False, 'seq_2M_encoder_units_2': 96, 'seq

### 개별 모델 및 Sequential 모델 성능 평가 함수 정의

In [25]:
@torch.no_grad()
def evaluate_individual_model(model, testloader, model_name):
    """개별 모델의 성능 평가"""
    model.eval()
    
    all_preds = []
    all_targets = []
    
    for batch in testloader:
        if model_name == "Model1":
            x_static, x_0M, x_0M_goutallier, y_2M = batch
            pred = model(x_static, x_0M, x_0M_goutallier)
            all_preds.append(pred)
            all_targets.append(y_2M)
        elif model_name == "Model2":
            x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
            pred = model(x_static, x_0M, x_2M, x_0M_goutallier)
            all_preds.append(pred)
            all_targets.append(y_3M)
        elif model_name == "Model3":
            x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
            pred = model(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
            all_preds.append(pred)
            all_targets.append(y_4M)
        elif model_name == "Model4":
            x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
            logits, regs, _ = model(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
            y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
            y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
            
            all_preds.append({'logits': logits, 'regs': regs})
            all_targets.append({'label': y_label, 'reg': y_reg})
    
    if model_name == "Model4":
        # 분류 및 회귀 성능 계산
        all_logits = torch.cat([p['logits'] for p in all_preds], dim=0)
        all_regs = torch.cat([p['regs'] for p in all_preds], dim=0)
        all_labels = torch.cat([t['label'] for t in all_targets], dim=0)
        all_reg_targets = torch.cat([t['reg'] for t in all_targets], dim=0)
        
        mse = F.mse_loss(all_regs, all_reg_targets).item()
        probs = all_logits.sigmoid().flatten()
        labels_int = all_labels.flatten().to(torch.int)
        
        # 분류 성능 계산
        pred_labels = (probs > 0.5).int().cpu().numpy()
        labels_np = labels_int.cpu().numpy()
        
        roc_auc = roc_auc_score(labels_np, probs.cpu().numpy())
        ap = average_precision_score(labels_np, probs.cpu().numpy())
        accuracy = accuracy_score(labels_np, pred_labels)
        
        return {
            'mse': mse,
            'roc_auc': roc_auc,
            'ap': ap,
            'accuracy': accuracy
        }
    else:
        # 회귀 성능만 계산
        all_preds = torch.cat(all_preds, dim=0)
        all_targets = torch.cat(all_targets, dim=0)
        mse = F.mse_loss(all_preds, all_targets).item()
        mae = F.l1_loss(all_preds, all_targets).item()
        
        return {
            'mse': mse,
            'mae': mae
        }

@torch.no_grad()
def evaluate_sequential_model(sequential_model, testset_model1):
    """Sequential 모델의 성능 평가 (0M 입력만으로 전체 시계열 예측)"""
    sequential_model.eval()
    
    all_pred_2M = []
    all_pred_3M = []
    all_pred_4M = []
    all_pred_6M = []
    all_pred_y_logits = []
    all_pred_6M_goutallier = []
    
    all_true_2M = []
    all_true_3M = []
    all_true_4M = []
    all_true_6M = []
    all_true_y = []
    all_true_6M_goutallier = []
    
    for i in range(len(testset_model1)):
        x_static, x_0M, x_0M_goutallier, y_2M = testset_model1[i]
        x_static = x_static.unsqueeze(0)
        x_0M = x_0M.unsqueeze(0)
        x_0M_goutallier = x_0M_goutallier.unsqueeze(0)
        
        # Sequential 모델 예측
        predictions = sequential_model(x_static, x_0M, x_0M_goutallier)
        
        all_pred_2M.append(predictions['pred_2M'])
        all_pred_3M.append(predictions['pred_3M'])
        all_pred_4M.append(predictions['pred_4M'])
        all_pred_6M.append(predictions['pred_6M'])
        all_pred_y_logits.append(predictions['pred_y_logits'])
        all_pred_6M_goutallier.append(predictions['pred_6M_goutallier'])
        
        # 실제 값 추출
        _, _, _, _, y_3M = testset_model2[i]
        _, _, _, _, _, y_4M = testset_model3[i]
        _, _, _, _, _, _, y_combined = testset_model4[i]
        
        y_6M = y_combined[:seq_features_6M]
        y_label = y_combined[seq_features_6M:seq_features_6M+1]
        y_6M_goutallier = y_combined[seq_features_6M+1:]
        
        all_true_2M.append(y_2M)
        all_true_3M.append(y_3M)
        all_true_4M.append(y_4M)
        all_true_6M.append(y_6M)
        all_true_y.append(y_label)
        all_true_6M_goutallier.append(y_6M_goutallier)
    
    # 텐서로 변환
    pred_2M = torch.cat(all_pred_2M, dim=0)
    pred_3M = torch.cat(all_pred_3M, dim=0)
    pred_4M = torch.cat(all_pred_4M, dim=0)
    pred_6M = torch.cat(all_pred_6M, dim=0)
    pred_y_logits = torch.cat(all_pred_y_logits, dim=0)
    pred_6M_goutallier = torch.cat(all_pred_6M_goutallier, dim=0)
    
    true_2M = torch.stack(all_true_2M)
    true_3M = torch.stack(all_true_3M)
    true_4M = torch.stack(all_true_4M)
    true_6M = torch.stack(all_true_6M)
    true_y = torch.stack(all_true_y)
    true_6M_goutallier = torch.stack(all_true_6M_goutallier)
    
    # 성능 계산
    mse_2M = F.mse_loss(pred_2M, true_2M).item()
    mse_3M = F.mse_loss(pred_3M, true_3M).item()
    mse_4M = F.mse_loss(pred_4M, true_4M).item()
    mse_6M = F.mse_loss(pred_6M, true_6M).item()
    mse_6M_goutallier = F.mse_loss(pred_6M_goutallier, true_6M_goutallier).item()
    
    mae_2M = F.l1_loss(pred_2M, true_2M).item()
    mae_3M = F.l1_loss(pred_3M, true_3M).item()
    mae_4M = F.l1_loss(pred_4M, true_4M).item()
    mae_6M = F.l1_loss(pred_6M, true_6M).item()
    mae_6M_goutallier = F.l1_loss(pred_6M_goutallier, true_6M_goutallier).item()
    
    # 분류 성능
    pred_y_probs = pred_y_logits.sigmoid().flatten()
    true_y_int = true_y.flatten().to(torch.int)
    
    # 분류 예측 및 성능 계산
    pred_labels = (pred_y_probs > 0.5).int().cpu().numpy()
    labels_np = true_y_int.cpu().numpy()
    
    roc_auc = roc_auc_score(labels_np, pred_y_probs.cpu().numpy())
    ap = average_precision_score(labels_np, pred_y_probs.cpu().numpy())
    accuracy = accuracy_score(labels_np, pred_labels)
    
    return {
        'mse_2M': mse_2M,
        'mse_3M': mse_3M,
        'mse_4M': mse_4M,
        'mse_6M': mse_6M,
        'mse_6M_goutallier': mse_6M_goutallier,
        'mae_2M': mae_2M,
        'mae_3M': mae_3M,
        'mae_4M': mae_4M,
        'mae_6M': mae_6M,
        'mae_6M_goutallier': mae_6M_goutallier,
        'roc_auc': roc_auc,
        'ap': ap,
        'accuracy': accuracy
    }

print("성능 평가 함수 정의 완료")


성능 평가 함수 정의 완료


### 최적화된 파라미터로 최종 모델 생성 및 학습

In [ ]:
def create_final_model_from_study(study, model_class, model_kwargs):
    """최적화된 파라미터로 최종 모델 생성"""
    best_trial = study.best_trial
    model = model_class(trial=best_trial, **model_kwargs)
    return model, best_trial.params

# 최적화된 모델 생성
print("\n" + "=" * 80)
print("최적화된 파라미터로 최종 모델 생성")
print("=" * 80)

# Model 1 최종 모델
print("\n[Model 1] 최종 모델 생성 중...")
final_model1, best_params1 = create_final_model_from_study(
    study1,
    OptimizedSequentialMLP1,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_2M': seq_features_2M
    }
)

# Model 2 최종 모델
print("[Model 2] 최종 모델 생성 중...")
final_model2, best_params2 = create_final_model_from_study(
    study2,
    OptimizedSequentialMLP2,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'seq_2M_features': seq_features_2M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_3M': seq_features_3M
    }
)

# Model 3 최종 모델
print("[Model 3] 최종 모델 생성 중...")
final_model3, best_params3 = create_final_model_from_study(
    study3,
    OptimizedSequentialMLP3,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'seq_2M_features': seq_features_2M,
        'seq_3M_features': seq_features_3M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_4M': seq_features_4M
    }
)

# Model 4 최종 모델
print("[Model 4] 최종 모델 생성 중...")
final_model4, best_params4 = create_final_model_from_study(
    study4,
    OptimizedSequentialMLP4,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'seq_2M_features': seq_features_2M,
        'seq_3M_features': seq_features_3M,
        'seq_4M_features': seq_features_4M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_total': seq_features_6M + 1 + goutallier_features_6M
    }
)

print("\n최종 모델 생성 완료!")

# 최종 모델 학습
print("\n" + "=" * 80)
print("최종 모델 학습 시작")
print("=" * 80)

# 최적 배치 크기 추출
batch_size1 = best_params1.get('batch_size', 64)
batch_size2 = best_params2.get('batch_size', 64)
batch_size3 = best_params3.get('batch_size', 64)
batch_size4 = best_params4.get('batch_size', 64)

# Model 1 학습
print("\n[Model 1] 학습 시작...")
trainloader1_final = DataLoader(trainset_model1, batch_size=batch_size1, shuffle=True, pin_memory=True)
valloader1_final = DataLoader(valset_model1, batch_size=batch_size1)
testloader1_final = DataLoader(testset_model1, batch_size=batch_size1)

trainer1_final = L.Trainer(
    max_epochs=100,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='final-model1-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model1-best'),
        EarlyStopping(monitor='val/loss', mode='min', patience=15)
    ]
)

trainer1_final.fit(final_model1, trainloader1_final, valloader1_final)
test_result1_final = trainer1_final.test(final_model1, testloader1_final)
print(f"[Model 1] 학습 완료 - Test MSE: {test_result1_final[0]['test/mse']:.6f}")

# Model 2 학습
print("\n[Model 2] 학습 시작...")
trainloader2_final = DataLoader(trainset_model2, batch_size=batch_size2, shuffle=True, pin_memory=True)
valloader2_final = DataLoader(valset_model2, batch_size=batch_size2)
testloader2_final = DataLoader(testset_model2, batch_size=batch_size2)

trainer2_final = L.Trainer(
    max_epochs=100,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='final-model2-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model2-best'),
        EarlyStopping(monitor='val/loss', mode='min', patience=15)
    ]
)

trainer2_final.fit(final_model2, trainloader2_final, valloader2_final)
test_result2_final = trainer2_final.test(final_model2, testloader2_final)
print(f"[Model 2] 학습 완료 - Test MSE: {test_result2_final[0]['test/mse']:.6f}")

# Model 3 학습
print("\n[Model 3] 학습 시작...")
trainloader3_final = DataLoader(trainset_model3, batch_size=batch_size3, shuffle=True, pin_memory=True)
valloader3_final = DataLoader(valset_model3, batch_size=batch_size3)
testloader3_final = DataLoader(testset_model3, batch_size=batch_size3)

trainer3_final = L.Trainer(
    max_epochs=100,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='final-model3-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model3-best'),
        EarlyStopping(monitor='val/loss', mode='min', patience=15)
    ]
)

trainer3_final.fit(final_model3, trainloader3_final, valloader3_final)
test_result3_final = trainer3_final.test(final_model3, testloader3_final)
print(f"[Model 3] 학습 완료 - Test MSE: {test_result3_final[0]['test/mse']:.6f}")

# Model 4 학습
print("\n[Model 4] 학습 시작...")
trainloader4_final = DataLoader(trainset_model4, batch_size=batch_size4, shuffle=True, pin_memory=True)
valloader4_final = DataLoader(valset_model4, batch_size=batch_size4)
testloader4_final = DataLoader(testset_model4, batch_size=batch_size4)

trainer4_final = L.Trainer(
    max_epochs=100,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/roc', mode='max', save_top_k=1, save_last=False, filename='final-model4-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model4-best'),
        EarlyStopping(monitor='val/roc', mode='max', patience=15)
    ]
)

trainer4_final.fit(final_model4, trainloader4_final, valloader4_final)
test_result4_final = trainer4_final.test(final_model4, testloader4_final)
print(f"[Model 4] 학습 완료 - Test ROC: {test_result4_final[0]['test/roc']:.6f}, Test AP: {test_result4_final[0]['test/ap']:.6f}")

print("\n" + "=" * 80)
print("모든 최종 모델 학습 완료!")
print("=" * 80)


Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
/data/miniconda3/envs/arcr/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name                  | Type           


최적화된 파라미터로 최종 모델 생성

[Model 1] 최종 모델 생성 중...
[Model 2] 최종 모델 생성 중...
[Model 3] 최종 모델 생성 중...
[Model 4] 최종 모델 생성 중...

최종 모델 생성 완료!

최종 모델 학습 시작

[Model 1] 학습 시작...
Epoch 0:  25%|██▍       | 14/57 [00:00<00:00, 97.18it/s, v_num=0, train/loss_step=1.030]

Epoch 27: 100%|██████████| 57/57 [00:00<00:00, 87.70it/s, v_num=0, train/loss_step=0.667, val/loss=1.030, train/loss_epoch=0.695]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/data/miniconda3/envs/arcr/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 110.19it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/loss           0.9623681902885437
        test/mse            0.9623681902885437
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


...


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 124.96it/s]

Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

   | Name                  | Type                   | Params | Mode 
--------------------------------------------------------------------------
0  | static_encoder        | Sequential             | 18.4 K | train
1  | seq_0M_encoder        | Sequential             | 38.9 K | train
2  | seq_2M_encoder        | Sequential             | 13.4 K | train
3  | seq_3M_encoder        | Sequential             | 49.6 K | train
4  | seq_4M_encoder        | Sequential             | 61.5 K | train
5  | goutallier_0M_encoder | Sequential             | 4.8 K  | train
6  | clshead             


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/loss           1.3153107166290283
        test/mse            1.3153107166290283
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Model 3] 학습 완료 - Test MSE: 1.315311

[Model 4] 학습 시작...
Epoch 45: 100%|██████████| 113/113 [00:02<00:00, 55.39it/s, v_num=3, train/loss_step=0.687, val/loss=0.665, train/loss_epoch=0.669]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 81.51it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test/ap            0.7708374261856079
      test/clf_loss         0.5909584760665894
        test/loss            0.668912410736084
        test/mse            1.2943609952926636
      test/reg_loss         0.38976967334747314
        test/roc            0.7491999864578247
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Model 4] 학습 완료 - Test ROC: 0.749200, Test AP: 0.770837

모든 최종 모델 학습 완료!


### Sequential 모델 연결 및 성능 평가

In [27]:
print("\n" + "=" * 80)
print("최적화된 Sequential 모델 생성 및 성능 평가")
print("=" * 80)

# Sequential 모델 생성
sequential_model = SequentialModel(final_model1, final_model2, final_model3, final_model4)
print("Sequential 모델 생성 완료")

# Test DataLoader 생성
batch_size = 64
testloader1 = DataLoader(testset_model1, batch_size=batch_size)
testloader2 = DataLoader(testset_model2, batch_size=batch_size)
testloader3 = DataLoader(testset_model3, batch_size=batch_size)
testloader4 = DataLoader(testset_model4, batch_size=batch_size)

# 개별 모델 성능 평가
print("\n" + "-" * 80)
print("개별 모델 성능 평가")
print("-" * 80)

print("\n[Model 1] 평가 중...")
model1_results = evaluate_individual_model(final_model1, testloader1, "Model1")
print(f"  MSE: {model1_results['mse']:.6f}")
print(f"  MAE: {model1_results['mae']:.6f}")

print("\n[Model 2] 평가 중...")
model2_results = evaluate_individual_model(final_model2, testloader2, "Model2")
print(f"  MSE: {model2_results['mse']:.6f}")
print(f"  MAE: {model2_results['mae']:.6f}")

print("\n[Model 3] 평가 중...")
model3_results = evaluate_individual_model(final_model3, testloader3, "Model3")
print(f"  MSE: {model3_results['mse']:.6f}")
print(f"  MAE: {model3_results['mae']:.6f}")

print("\n[Model 4] 평가 중...")
model4_results = evaluate_individual_model(final_model4, testloader4, "Model4")
print(f"  MSE: {model4_results['mse']:.6f}")
print(f"  ROC AUC: {model4_results['roc_auc']:.6f}")
print(f"  AP: {model4_results['ap']:.6f}")
print(f"  Accuracy: {model4_results['accuracy']:.6f}")

# Sequential 모델 성능 평가
print("\n" + "-" * 80)
print("Sequential 모델 성능 평가 (0M 입력만으로 전체 시계열 예측)")
print("-" * 80)

sequential_results = evaluate_sequential_model(sequential_model, testset_model1)

print("\n[회귀 성능 - MSE]")
print(f"  2M 예측 MSE: {sequential_results['mse_2M']:.6f}")
print(f"  3M 예측 MSE: {sequential_results['mse_3M']:.6f}")
print(f"  4M 예측 MSE: {sequential_results['mse_4M']:.6f}")
print(f"  6M 예측 MSE: {sequential_results['mse_6M']:.6f}")
print(f"  6M Goutallier 예측 MSE: {sequential_results['mse_6M_goutallier']:.6f}")

print("\n[회귀 성능 - MAE]")
print(f"  2M 예측 MAE: {sequential_results['mae_2M']:.6f}")
print(f"  3M 예측 MAE: {sequential_results['mae_3M']:.6f}")
print(f"  4M 예측 MAE: {sequential_results['mae_4M']:.6f}")
print(f"  6M 예측 MAE: {sequential_results['mae_6M']:.6f}")
print(f"  6M Goutallier 예측 MAE: {sequential_results['mae_6M_goutallier']:.6f}")

print("\n[분류 성능]")
print(f"  ROC AUC: {sequential_results['roc_auc']:.6f}")
print(f"  AP (Average Precision): {sequential_results['ap']:.6f}")
print(f"  Accuracy: {sequential_results['accuracy']:.6f}")

# 성능 요약
print("\n" + "=" * 80)
print("성능 평가 요약")
print("=" * 80)
print("\n개별 모델 성능:")
print(f"  Model 1 (2M 예측): MSE={model1_results['mse']:.6f}, MAE={model1_results['mae']:.6f}")
print(f"  Model 2 (3M 예측): MSE={model2_results['mse']:.6f}, MAE={model2_results['mae']:.6f}")
print(f"  Model 3 (4M 예측): MSE={model3_results['mse']:.6f}, MAE={model3_results['mae']:.6f}")
print(f"  Model 4 (6M+분류): MSE={model4_results['mse']:.6f}, ROC={model4_results['roc_auc']:.6f}, AP={model4_results['ap']:.6f}, Accuracy={model4_results['accuracy']:.6f}")
print("\nSequential 모델 성능:")
print(f"  전체 파이프라인: ROC={sequential_results['roc_auc']:.6f}, AP={sequential_results['ap']:.6f}, Accuracy={sequential_results['accuracy']:.6f}")
print(f"  중간 단계 예측: 2M MSE={sequential_results['mse_2M']:.6f}, 3M MSE={sequential_results['mse_3M']:.6f}, 4M MSE={sequential_results['mse_4M']:.6f}, 6M MSE={sequential_results['mse_6M']:.6f}")

print("\n" + "=" * 80)
print("모든 평가 완료!")
print("=" * 80)



최적화된 Sequential 모델 생성 및 성능 평가
Sequential 모델 생성 완료

--------------------------------------------------------------------------------
개별 모델 성능 평가
--------------------------------------------------------------------------------

[Model 1] 평가 중...
  MSE: 0.962368
  MAE: 0.778129

[Model 2] 평가 중...
  MSE: 0.902871
  MAE: 0.674036

[Model 3] 평가 중...
  MSE: 1.315310
  MAE: 0.773468

[Model 4] 평가 중...
  MSE: 1.294361
  ROC AUC: 0.749200
  AP: 0.770837
  Accuracy: 0.680000

--------------------------------------------------------------------------------
Sequential 모델 성능 평가 (0M 입력만으로 전체 시계열 예측)
--------------------------------------------------------------------------------

[회귀 성능 - MSE]
  2M 예측 MSE: 0.962368
  3M 예측 MSE: 1.034777
  4M 예측 MSE: 1.688876
  6M 예측 MSE: 1.741515
  6M Goutallier 예측 MSE: 1.023012

[회귀 성능 - MAE]
  2M 예측 MAE: 0.778129
  3M 예측 MAE: 0.736314
  4M 예측 MAE: 0.890636
  6M 예측 MAE: 0.934148
  6M Goutallier 예측 MAE: 0.470817

[분류 성능]
  ROC AUC: 0.574000
  AP (Average Precision):